# Building DNN Models for Classification with TF core

Here we are using just a small subset of the data for demonstration pourposes. The complete dataset can be accessed here:
https://archive.ics.uci.edu/ml/machine-learning-databases/00280/HIGGS.csv.gz


In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os

from tensorflow.contrib.layers import l2_regularizer
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score

%matplotlib inline

## 1. Load and prepare the data

In [2]:
from numpy import genfromtxt
current_dir = os.getcwd()

## Training data
train_dataset_path = os.path.join(current_dir, os.pardir, 'data', 'small_higgs.csv')
higgs_train = genfromtxt(train_dataset_path, delimiter=',')
X_train = higgs_train[:,1:]
y_train = higgs_train[:,0]
del higgs_train

# Validation data
validation_dataset_path = os.path.join(os.getcwd(), os.pardir, 'data', 'validation_higgs.csv')
higgs_val = genfromtxt(validation_dataset_path, delimiter=',')
X_val = higgs_val[:,1:]
y_val = higgs_val[:,0]
del higgs_val

## 2. Building the input pipepline

In [3]:
BATCH_SIZE = 128

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=10000)
train_dataset = train_dataset.batch(BATCH_SIZE)
iterator = train_dataset.make_initializable_iterator()
next_element = iterator.get_next()

## 3. Build a function containing the DNN: use L2 regularization

Here we are using `tf.contrib.layers.l2_regularizer`, from the documentation:

*Args:*

scale: A scalar multiplier Tensor. 0.0 disables the regularizer.
scope: An optional scope name.

*Returns:*

A function with signature l2(weights) that applies L2 regularization.

In [4]:
SCALE_REGULARIZER = 0.01

n_hidden1 = 200
n_hidden2 = 200
n_hidden3 = 200
n_hidden4 = 200
n_outputs = 1

def DNN_L2_Reg(inputs, scale_reg=SCALE_REGULARIZER):
    wt_init = tf.contrib.layers.xavier_initializer()
    l2_reg = l2_regularizer(scale_reg)
    
    hidden1 = tf.layers.dense(inputs, units=n_hidden1,
                              activation=tf.nn.elu, 
                              kernel_initializer=wt_init,
                              kernel_regularizer=l2_reg)

    hidden2 = tf.layers.dense(hidden1, units=n_hidden1,
                              activation=tf.nn.elu, 
                              kernel_initializer=wt_init,
                              kernel_regularizer=l2_reg)
    
    hidden3 = tf.layers.dense(hidden2, units=n_hidden1,
                              activation=tf.nn.elu, 
                              kernel_initializer=wt_init,
                              kernel_regularizer=l2_reg)
        
    hidden4 = tf.layers.dense(hidden3, units=n_hidden1,
                              activation=tf.nn.elu, 
                              kernel_initializer=wt_init,
                              kernel_regularizer=l2_reg)
    
    logits = tf.layers.dense(hidden4, units=n_outputs, activation=None)
    return tf.squeeze(logits)

## 4. Create the placeholders to pass values for training and evaluation

In [5]:
n_inputs = X_train.shape[1] # number of features in the dataset
X = tf.placeholder(tf.float32, shape=[None, n_inputs], name='X')
y = tf.placeholder(tf.float32, name='target')

## 5. Define the loss: add the proper regularization terms to the loss

In [6]:
logits = DNN_L2_Reg(X)
cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(cross_entropy)

## Getting the values regularization weigths
L2_weights = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
sum_L2_weights = tf.reduce_sum(L2_weights)
loss += sum_L2_weights

tf.summary.scalar('loss', loss)

## Optional: recording some metrics for visualization
prob_of_signal = tf.nn.sigmoid(logits)
y_pred = tf.cast(prob_of_signal > 0.5, dtype=tf.int32)
accuracy, accuracy_update_op= tf.metrics.accuracy(labels=y, predictions=y_pred)
tf.summary.scalar('accuracy', accuracy)
auc, auc_update_op = tf.metrics.accuracy(labels=y, predictions=y_pred)
tf.summary.scalar('AUC', auc)

## Summary an writer objects for TensorBoard
summary_values = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(os.path.join(current_dir, 'higgs_logs','L2_Reg_train'))
val_writer = tf.summary.FileWriter(os.path.join(current_dir, 'higgs_logs','L2_Reg_validation'))

## 6. Define the optimizer and training operation


In [7]:
optimizer = tf.train.AdamOptimizer()
training_op = optimizer.minimize(loss)

## 7. (Optional) Write a function for running the training operation

In [8]:
def train_model(epoch_number):
    print(epoch_number, end=',')
    iterator.initializer.run()
    ## This is necesary for the metrics:
    tf.local_variables_initializer().run()
    while True:
        try:
            X_values, y_values = sess.run(next_element)
            sess.run([training_op, accuracy_update_op, auc_update_op], 
                     feed_dict={X: X_values, y:y_values})
        except tf.errors.OutOfRangeError:
            break
    ## Training metrics
    summaries = sess.run(summary_values, feed_dict={X:X_values, y:y_values})
    train_writer.add_summary(summaries, epoch_number)
    ## The values for the metrics must be re-initialized
    tf.local_variables_initializer().run()
    sess.run([accuracy_update_op, auc_update_op], feed_dict={X: X_val, y:y_val})
    summaries = sess.run(summary_values, feed_dict={X:X_val, y:y_val})
    val_writer.add_summary(summaries, epoch_number)

## 8. Run the computation graph

In [9]:
N_EPOCHS = 400
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    train_writer.add_graph(sess.graph)
    ## Training loop
    print("Epoch: ")
    for epoch in range(1,N_EPOCHS+1):
        train_model(epoch)
    print("\nDone Training!")
    
    # Closing the file writers
    train_writer.close()
    val_writer.close()
    # Getting predictions
    predictions = sess.run(y_pred, feed_dict={X: X_val})

Epoch: 
1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,

## 9. Visualize/analyze the results of the model

In [10]:
confusion_matrix(y_true=y_val, y_pred=predictions)

array([[1343,  982],
       [ 669, 2006]])

In [11]:
print("Precision: ", precision_score(y_true=y_val, y_pred=predictions))
print("Recall: ", recall_score(y_true=y_val, y_pred=predictions))
print("Accuracy: ", accuracy_score(y_true=y_val, y_pred=predictions))

Precision:  0.671352074967
Recall:  0.749906542056
Accuracy:  0.6698
